In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.4/904.4 kB 20.1 MB/s eta 0:00:00


In [2]:
import os
import glob
from PIL import Image
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Example usage:
BKAI_train_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/train_images'
BKAI_labels_folder = '/kaggle/input/bkai-dataset2/vietnamese/labels'

BKAI_val_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/test_image'

BKAI_test_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/unseen_test_images'

BKAI_train_output_images_folder = "/kaggle/working/train/images"
BKAI_train_output_labels_folder = "/kaggle/working/train/labels"

BKAI_val_output_images_folder = "/kaggle/working/val/images"
BKAI_val_output_labels_folder = "/kaggle/working/val/labels"

BKAI_test_output_images_folder = "/kaggle/working/test/images"
BKAI_test_output_labels_folder = "/kaggle/working/test/labels"

# Define paths
BKAI1_train_images_folder = '/kaggle/input/bkai-dataset-1/train_imgs/training_img'
BKAI1_train_labels_folder = '/kaggle/input/bkai-dataset-1/train_gt/training_gt'
BKAI1_train_output_images_folder = "/kaggle/working/train/images"
BKAI1_train_output_labels_folder = "/kaggle/working/train/labels"

In [4]:
import os
import shutil
from PIL import Image

def is_image_valid(image_path):
    """
    Kiểm tra ảnh có hợp lệ không (có thể mở được).
    """
    try:
        with Image.open(image_path) as img:
            img.verify()  # Kiểm tra tính toàn vẹn của ảnh
        return True
    except Exception as e:
        print(f"Invalid image: {image_path}, Error: {e}")
        return False

def is_bbox_valid(x_min, y_min, x_max, y_max, img_width, img_height):
    """
    Kiểm tra xem bbox có nằm trong giới hạn của ảnh không và có hợp lệ không.
    """
    return 0 <= x_min < x_max <= img_width and 0 <= y_min < y_max <= img_height

def convert_to_yolo_format(input_labels_folder, input_images_folder, 
                           output_labels_folder, output_images_folder):
    if not os.path.exists(output_labels_folder):
        os.makedirs(output_labels_folder)
    if not os.path.exists(output_images_folder):
        os.makedirs(output_images_folder)

    for label_file in os.listdir(input_labels_folder):
        if label_file.endswith('.txt'):
            label_path = os.path.join(input_labels_folder, label_file)
            output_label_path = os.path.join(output_labels_folder, label_file)

            # Lấy file ảnh tương ứng để xác định kích thước
            image_file = label_file.replace('gt_', '').replace('.txt', '.jpg')
            new_label_name = image_file.replace('.jpg', '.txt')
            image_path = os.path.join(input_images_folder, image_file)

            if not os.path.exists(image_path) or not is_image_valid(image_path):
                continue

            with Image.open(image_path) as img:
                image_width, image_height = img.size

            yolo_labels = []  # Lưu các bbox hợp lệ
            with open(label_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split(',')

                # Đảm bảo đủ tọa độ bbox (ít nhất 8 giá trị)
                if len(parts) < 8:
                    continue

                try:
                    # Lấy tọa độ bbox
                    x1, y1, x2, y2, x3, y3, x4, y4 = map(float, parts[:8])
                    x_min = min(x1, x2, x3, x4)
                    y_min = min(y1, y2, y3, y4)
                    x_max = max(x1, x2, x3, x4)
                    y_max = max(y1, y2, y3, y4)

                    # Kiểm tra bbox hợp lệ
                    if not is_bbox_valid(x_min, y_min, x_max, y_max, image_width, image_height):
                        continue

                    # Tính tọa độ YOLO format
                    bbox_width = (x_max - x_min) / image_width
                    bbox_height = (y_max - y_min) / image_height
                    x_center = (x_min + x_max) / 2 / image_width
                    y_center = (y_min + y_max) / 2 / image_height

                    # Gán class_id là 0
                    class_id = 0

                    yolo_labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")
                except ValueError:
                    continue  # Bỏ qua dòng không hợp lệ

            # Lưu bbox nếu có bbox hợp lệ
            if yolo_labels:
                with open(os.path.join(output_labels_folder, new_label_name), 'w') as out_f:
                    out_f.write("\n".join(yolo_labels))

                # Sao chép ảnh sang thư mục đầu ra
                shutil.copy(image_path, os.path.join(output_images_folder, image_file))

In [5]:
# Call the function
convert_to_yolo_format(BKAI1_train_labels_folder, BKAI1_train_images_folder, 
                       BKAI1_train_output_labels_folder, BKAI1_train_output_images_folder)

# BKAI 2


In [6]:
import os
import shutil
import cv2
from PIL import Image

def ensure_directories_exist(directories):
    """
    Ensure that all directories in the given list exist.
    """
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

def convert_bbox_to_yolo(bbox, img_width, img_height):
    """
    Convert bounding box from absolute coordinates to YOLO format.
    bbox: [x1, y1, x2, y2, x3, y3, x4, y4]
    """
    x_coords = bbox[0::2]  # [x1, x2, x3, x4]
    y_coords = bbox[1::2]  # [y1, y2, y3, y4]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)
    
    x_center = (x_min + x_max) / 2.0 / img_width
    y_center = (y_min + y_max) / 2.0 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    
    return x_center, y_center, width, height

def is_bbox_valid(bbox, img_width, img_height):
    """
    Check if the bounding box is within the image boundaries and has valid dimensions.
    bbox: [x1, y1, x2, y2, x3, y3, x4, y4]
    """
    x_coords = bbox[0::2]
    y_coords = bbox[1::2]
    
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)
    
    # Check if the bbox is within the image boundaries
    return 0 <= x_min < x_max <= img_width and 0 <= y_min < y_max <= img_height

def is_image_valid(image_path):
    """
    Check if the image is valid (can be opened).
    """
    try:
        # Use OpenCV to check if the image can be read
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError("Image could not be read with OpenCV")
        
        # Alternatively, use PIL to ensure the image is not corrupt
        img_pil = Image.open(image_path)
        img_pil.verify()  # Verify the image integrity
        return True
    except Exception as e:
        print(f"Error with image {image_path}: {e}")
        return False

def process_labels_and_images(
    images_folder, labels_folder, output_images_folder, output_labels_folder
):
    """
    Process images and their corresponding label files, converting bounding boxes to YOLO format.
    """
    ensure_directories_exist([output_images_folder, output_labels_folder])
    
    for image_name in os.listdir(images_folder):
        if not image_name.endswith('.jpg'):
            continue

        # Map image to label
        image_id = str(int(image_name.split('.')[0].replace("im", "")))  # e.g., im0001 -> 1
        label_file = os.path.join(labels_folder, f"gt_{image_id}.txt")
        if not os.path.exists(label_file):
            continue

        # Read image dimensions
        image_path = os.path.join(images_folder, image_name)
        
        # Check if image is valid
        if not is_image_valid(image_path):
            continue  # Skip the image if it's corrupt

        img = cv2.imread(image_path)
        img_height, img_width = img.shape[:2]

        # Process label file
        yolo_labels = []
        for line in open(label_file, 'r', encoding='utf-8'):
            parts = line.strip().split(',')
            bbox = list(map(int, parts[:8]))  # First 8 values are bbox
            label = parts[8]  # Last value is the text/label

            # Check if the bbox is valid
            if is_bbox_valid(bbox, img_width, img_height):
                # Convert bbox to YOLO format
                x_center, y_center, width, height = convert_bbox_to_yolo(bbox, img_width, img_height)
                yolo_labels.append(f"0 {x_center} {y_center} {width} {height}")
        
        if not yolo_labels:  # Skip image if no valid bbox
            continue
        
        # Save image to output folder
        new_image_name = f"{image_id}.jpg"
        shutil.copy(image_path, os.path.join(output_images_folder, new_image_name))

        # Save YOLO labels to output folder
        output_label_file = os.path.join(output_labels_folder, f"{image_id}.txt")
        with open(output_label_file, 'w') as f:
            f.write("\n".join(yolo_labels))

In [7]:
process_labels_and_images(
    BKAI_train_images_folder, 
    BKAI_labels_folder, 
    BKAI_train_output_images_folder, 
    BKAI_train_output_labels_folder
)

In [8]:
process_labels_and_images(
    BKAI_val_images_folder, 
    BKAI_labels_folder, 
    BKAI_val_output_images_folder, 
    BKAI_val_output_labels_folder
)

In [9]:
process_labels_and_images(
    BKAI_test_images_folder, 
    BKAI_labels_folder, 
    BKAI_test_output_images_folder, 
    BKAI_test_output_labels_folder
)

In [10]:
# Tạo đường dẫn tới file YAML
dataset_dir = "/kaggle/working"
os.makedirs(dataset_dir, exist_ok=True)
yaml_file = os.path.join(dataset_dir, "data.yaml")

# Nội dung file YAML
yaml_content = """
train: /kaggle/working/train/images
val: /kaggle/working/val/images

nc: 1  # Số nhãn (với OCR chỉ là 1 lớp văn bản)
names: ['text']
"""

# Ghi nội dung vào file YAML
with open(yaml_file, "w") as f:
    f.write(yaml_content.strip())

print(f"File 'data.yaml' đã được tạo tại: {yaml_file}")

File 'data.yaml' đã được tạo tại: /kaggle/working/data.yaml


In [11]:
# Load a COCO-pretrained YOLOv8n model
model = YOLO("/kaggle/input/train-lai-step-1-uit-ver-2/pytorch/default/1/best.pt")

# Display model information (optional)
model.info()

Model summary: 365 layers, 68,153,571 parameters, 0 gradients, 258.1 GFLOPs


(365, 68153571, 0, 258.1225472)

In [12]:
results = model.train(
    data="/kaggle/working/data.yaml",  
    epochs=150,                        
    imgsz=640,                         
    optimizer="AdamW",                 
    patience=15,                       
    project="YOLOv8 - Step 2 - AdamW", 
    name="Optimizer with conf 1",                 
    batch=32,   
    momentum=0.85,
    lr0=0.00001,                       
    lrf=0.01,                          
    weight_decay=0.001,               
    seed=42,                           
    device=[0, 1],                     
    cos_lr=True,
    warmup_epochs = 10
)

Ultralytics 8.3.56 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/train-lai-step-1-uit-ver-2/pytorch/default/1/best.pt, data=/kaggle/working/data.yaml, epochs=150, time=None, patience=15, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=YOLOv8 - Step 2 - AdamW, name=Optimizer with conf 1, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes

100%|██████████| 755k/755k [00:00<00:00, 23.2MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics.nn.modules.conv.Conv             [640

100%|██████████| 5.35M/5.35M [00:00<00:00, 117MB/s]


AMP: checks passed ✅


...

                   all        300       8649      0.855      0.655      0.723      0.499

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     62/150      14.2G      1.017     0.4881       0.97        110        640: 100%|██████████| 54/54 [01:25<00:00,  1.59s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:09<00:00,  1.03it/s]


                   all        300       8649      0.855      0.655      0.725      0.505
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 47, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

62 epochs completed in 1.677 hours.
Optimizer stripped from YOLOv8 - Step 2 - AdamW/Optimizer with conf 1/weights/last.pt, 136.7MB
Optimizer stripped from YOLOv8 - Step 2 - AdamW/Optimizer with conf 1/weights/best.pt, 136.7MB

Validating YOLOv8 - Step 2 - AdamW/Optimizer with conf 1/weights/best.pt...
Ultralytics 8.3.56 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:11<00:00,  1.16s/it]


                   all        300       8649      0.858      0.653      0.724      0.507
Speed: 0.1ms preprocess, 26.4ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to YOLOv8 - Step 2 - AdamW/Optimizer with conf 1
